In [4]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 27.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 14.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 25.6 MB/s eta 0:00:00


In [5]:
import random
import json
import getpass
from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)
import pandas as pd
from shapely.geometry import Polygon
import requests

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

# Load your data
def inference(df):
    X = df[['ndvi_mean', 'ndwi_mean', 'avg_elevation']]
    with open('scaler.pkl', 'rb') as file:
        scaler = pickle.load(file)
    X = scaler.transform(X)
    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)
    y_pred_probs = model.predict_proba(X)[:, 1]
    return y_pred_probs


In [7]:
config = SHConfig()
config.sh_client_id = getpass.getpass("sh-45e9da6d-83d3-4b94-863f-fdfbf42353ac")
config.sh_client_secret = getpass.getpass("1psSm3NyziCaifLzZKsFBOjLFqMlzIHx")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")

sh-45e9da6d-83d3-4b94-863f-fdfbf42353ac ········
1psSm3NyziCaifLzZKsFBOjLFqMlzIHx ········


In [8]:
# define functions to extract statistics for all acquisition dates
def extract_stats(date, stat_data):
    d = {}
    for key, value in stat_data["outputs"].items():
        stats = value["bands"]["B0"]["stats"]
        if stats["sampleCount"] == stats["noDataCount"]:
            continue
        else:
            d["date"] = [date]
            for stat_name, stat_value in stats.items():
                if stat_name == "sampleCount" or stat_name == "noDataCount":
                    continue
                else:
                    d[f"{key}_{stat_name}"] = [stat_value]
    return pd.DataFrame(d)


def read_acquisitions_stats(stat_data):
    df_li = []
    for aq in stat_data:
        date = aq["interval"]["from"][:10]
        df_li.append(extract_stats(date, aq))
    return pd.concat(df_li)

In [9]:
evalscript_ndvi = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B04",
        "B08",
        "dataMask"
      ]
    }],
    output: [
      {
        id: "ndvi",
        bands: 1
      },
      {
        id: "dataMask",
        bands: 1
      }]
  };
}

function evaluatePixel(samples) {
    let index = (samples.B08 - samples.B04) / (samples.B08+samples.B04);
    return {
        ndvi: [index],
        dataMask: [samples.dataMask],
    };
}

"""

evalscript_ndwi = """
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B03", // Green band
        "B08", // NIR band
        "dataMask" // Mask to exclude the non-valid pixels
      ]
    }],
    output: [
      {
        id: "ndwi",
        bands: 1,
        sampleType: "FLOAT32"
      },
      {
        id: "dataMask",
        bands: 1
      }
    ]
  };
}

function evaluatePixel(samples) {
    let ndwi = (samples.B03 - samples.B08) / (samples.B03 + samples.B08);
    return {
        ndwi: [ndwi],
        dataMask: [samples.dataMask],
    };
}
"""

In [10]:
def api_request(geojson_data):
    geometry = geojson_data['geometry']
    def get_open_elevation(lat, lon):
        query = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}'
        response = requests.get(query).json()
        # Check if the response contains results
        if 'results' in response:
            return response['results'][0]['elevation']
        else:
            return 270.0
    a = get_open_elevation(*geometry['coordinates'][0][0]) 
    
    # Create a Geometry object for the Sentinel Hub request
    polygon_geometry = Geometry(geometry=geometry, crs=CRS.WGS84)

    request_ndvi = SentinelHubStatistical(
        aggregation=SentinelHubStatistical.aggregation(
            evalscript=evalscript_ndvi,
            time_interval=("2020-01-01T00:00:00Z", "2020-12-30T23:59:59Z"),
            aggregation_interval="P1D",
            size=[200, 200],

        ),
        input_data=[
            SentinelHubStatistical.input_data(
                DataCollection.SENTINEL2_L1C.define_from(
                    name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
                ),
                other_args={"dataFilter": {"maxCloudCoverage": 10}},
            ),
        ],
        geometry=polygon_geometry,
        config=config,
    )
    request_ndwi = SentinelHubStatistical(
        aggregation=SentinelHubStatistical.aggregation(
            evalscript=evalscript_ndwi,
            time_interval=("2020-01-01T00:00:00Z", "2020-12-30T23:59:59Z"),
            aggregation_interval="P1D",
            size=[200, 200],
        ),
        input_data=[
            SentinelHubStatistical.input_data(
                DataCollection.SENTINEL2_L1C.define_from(
                    name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
                ),
                other_args={"dataFilter": {"maxCloudCoverage": 10}},
            ),
        ],
        geometry=polygon_geometry,
        config=config,
    )
    response_ndvi = request_ndvi.get_data()
    response_ndwi = request_ndwi.get_data()
    # result_ndvi = read_acquisitions_stats(response_ndvi[0]["data"])
    # result_ndwi = read_acquisitions_stats(response_ndwi[0]["data"])
    # Assume response_ndvi and response_ndwi are now populated with data
    # Convert results to DataFrame
    df_ndvi = pd.DataFrame(read_acquisitions_stats(response_ndvi[0]['data']))
    df_ndwi = pd.DataFrame(read_acquisitions_stats(response_ndwi[0]['data']))

    # Convert the 'date' column to datetime format to ease processing
    df_ndvi['date'] = pd.to_datetime(df_ndvi['date'])
    df_ndwi['date'] = pd.to_datetime(df_ndwi['date'])

    # Group by year and calculate the mean of the means for each year
    yearly_ndvi = df_ndvi.groupby(df_ndvi['date'].dt.year)['ndvi_mean'].mean().reset_index()
    yearly_ndwi = df_ndwi.groupby(df_ndwi['date'].dt.year)['ndwi_mean'].mean().reset_index()

    # Combine the yearly means into a single dataframe
    yearly_combined = pd.merge(yearly_ndvi, yearly_ndwi, on='date', how='outer', suffixes=('_ndvi', '_ndwi'))

    yearly_combined['avg_elevation'] = a  # Assuming average_elevations list is already populated

    # Rename the columns to reflect the data correctly
    yearly_combined.rename(columns={'date': 'year'}, inplace=True)

    return yearly_combined

In [11]:
def ai(geo_json):
    datapoint_df = api_request(geo_json)
    probability = inference(datapoint_df)
    #normalize           
    #lg.regression predict
    return probability[0]

In [12]:
def float_to_color(value):
    if not (0 <= value <= 1):
        raise ValueError("Value must be within [0, 1].")

    # Linearly interpolate between green (0,255,0) and red (255,0,0)
    red = int(255 * value)
    green = int(255 * (1 - value))
    blue = 0  # No blue component
    
    return '#{:02X}{:02X}{:02X}'.format(red, green, blue)

In [16]:
from ipyleaflet import Map, DrawControl, GeoJSON
import random
import json

# Function to generate a random color in hexadecimal format
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Create a Map instance
m = Map(center=(41.3874, 2.1686), zoom=4)

# Create a DrawControl instance
draw_control = DrawControl()

# Only allow Polygon drawing
draw_control.polygon = {
    "shapeOptions": {
        "fillOpacity": 0.8,
        "color": "#333333"  # Default color, will be overridden by random color
    },
    "allowIntersection": False
}
draw_control.polyline = {}
draw_control.circlemarker = {}
draw_control.rectangle = {}
draw_control.circle = {}
draw_control.marker = {}

def handle_draw(target, action, geo_json):
    # When a polygon is created, set its color to a random value
    if action == 'created':
        print(geo_json)
        risk = ai(geo_json)
        color = float_to_color(risk)
        style = {
            'fillColor': color,
            'weight': 1,
            'color': color
        }
        geo_json['properties']['style'] = style
        geo_layer = GeoJSON(data=geo_json, style=style)
        m.add_layer(geo_layer)
        # save
        file_path = 'polygon.geojson'
        with open(file_path, 'w') as f:
            json.dump(geo_json, f)
        print(f'GeoJSON saved to {file_path}')



# Attach the draw handler to the draw_control
draw_control.on_draw(handle_draw)

# Add the draw control to the map
m.add_control(draw_control)

# Display the map
m


Map(center=[41.3874, 2.1686], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [14]:
!pip install ipyleaflet
!jupyter nbextension enable --py --sys-prefix ipyleaflet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 21.7 MB/s eta 0:00:00
Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK
